# Cеточные модели уравнений с частными производными

## Лабораторная №2 (Метод Рунге-Кутты)

Выполнил Мусатов Данила Юрьевич КМБО-03-18

# Задание №2

## Метод Рунге-Кутты
### Семейство явных методов Рунге-Кутты $p$-го порядка записывается в виде совокупности формул:

$$
\begin{gathered}
y_{k+1}=y_{k}+\Delta y_{k}, \quad y_{k}=\sum_{i=1}^{p} c_{k} K_{i}^{k} \\
K_{i}^{k}=h f\left(x_{k}+a_{i} h, y_{k}+h \sum_{j=1}^{i-1} b_{i j} K_{j}^{k}\right), \quad i=2,3, \ldots, p
\end{gathered}
$$

Параметры $a_{i}, b_{i j}, c_{i}$ подбираются так, чтобы значение $y_{k+1}$ совпадало со значением разложения в точке $x_{k+1}$ точного решения в ряд Тейлора с погрешностю $O\left(h^{p+1}\right)$.


#### Метод Рунге-Кутты четвертого порядка

Имеем:

$$
\begin{gathered}
a_{1}=0, \quad a_{2}=\frac{1}{2}, \quad a_{3}=\frac{1}{2}, \quad a_{4}=1, \\
b_{21}=\frac{1}{2}, \quad b_{31}=0, \quad b_{32}=\frac{1}{2}, b_{41}=0, \quad b_{42}=0, \quad b_{43}=\frac{1}{2}, \\
c_{1}=\frac{1}{6}, \quad c_{2}=\frac{1}{3} \quad c_{3}=\frac{1}{3}, c_{4}=\frac{1}{6},
\end{gathered}
$$

тогда

$$
\begin{gathered}
y_{k+1}=y_{k}+\Delta y_{k}, \quad \Delta y_{k}=\frac{1}{6}\left(K_{1}^{k}+2 K_{2}^{k}+2 K_{3}^{k}+k_{4}^{k}\right) \\
K_{1}^{k}=h f\left(x_{k}, y_{k}\right), \quad K_{2}^{k}=h f\left(x_{k}+\frac{1}{2} h, y_{k}+\frac{1}{2} K_{1}^{k}\right) \\
K_{3}^{k}=h f\left(x_{k}+\frac{1}{2} h, y_{k}+\frac{1}{2} K_{2}^{k}\right), \quad K_{4}^{k}=h f\left(x_{k}+h, y_{k}+K_{3}^{k}\right)
\end{gathered}
$$

#### Задание:
Решить задачу Коши методом Рунге-Кутты 4-го порядка со значениями шага $h_{1}=0.1$ и $h_{2}=0.01$. Сравнить результаты графически.

#### Вариант 2: $\quad y^{\prime}=\frac{-y x+x\left(x^{2}+1\right)}{x^{2}+1}, \quad y(0)=1, \quad x \in[0,1]$ .

### Аналитическое решение

$$y{\left(0 \right)} = 1$$<br><br>$$y{\left(x \right)} = \frac{C_{1} \sqrt{x^{2} + 1} + x^{4} + 2 x^{2} + 1}{3 \left(x^{2} + 1\right)}$$<br><br>$$1 = \frac{C_{1} \sqrt{0^{2} + 1} + 0^{4} + 2 \cdot 0^{2} + 1}{3 \left(0^{2} + 1\right)}$$<br><br>$$C_{1} = 2$$<br><br>$$y{\left(x \right)} = \frac{x^{4} + 2 x^{2} + 2 \sqrt{x^{2} + 1} + 1}{3 \left(x^{2} + 1\right)}$$<br><br><a data-pod="funcgraphs.xy" data-input='{"function": "(1 + x^4 + 2*x^2 + 2*sqrt(1 + x^2))/(3*(1 + x^2))", "x": "x"}' style="display: none;" >Построить график</a>

## Численное решение

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as ip_w 

#Исходные данные
A, B = 0, 1
func = lambda x, y: (-y*x+x*(x**2 + 1))/(x**2 + 1) # определение исходной функции
source_func = lambda x: (x**4+2*(x**2)+2*np.sqrt(x**2+1)+1 )/ (3*(x**2 + 1)) # точное решение
Y0 = 1
h1 = 0.1
h2 = 0.01

In [2]:
def Method_Runge_Kutta(f,  y0, left, right, h):
    xs = np.arange(left, right+h, h) # определение точек x
    # определение коэффициентов K
    K1 = lambda x, y: func(x, y)
    K2 = lambda x, y, h: func(x + h/2, y + h*K1(x, y)/2)
    K3 = lambda x, y, h: func(x + h/2, y + h*K2(x, y, h)/2)
    K4 = lambda x, y, h: func(x + h, y + h*K3(x, y, h))
    y_next = y0
    ys= [y_next] # список с значениями y
    # рассчет согласно формулам
    for x in xs[1:]:
        y_next = y_next + (h/6) * (K1(x, y_next) + 2*K2(x, y_next, h) + 2*K3(x, y_next, h) + K4(x, y_next, h))
        ys.append(y_next) # добавление значения
    return xs, np.array(ys)

In [3]:
# Функция для оценки максимального отклонения от аналитического решения
def max_error(xs, ys, source_func):
    return max(np.fabs(ys - source_func(xs)))

In [4]:
def comparison_source_with_Method_Runge_Kutta(y0, a, b, h):
    xs, ys = Method_Runge_Kutta(func, y0, a, b, h)
    plt.rcParams["figure.figsize"] = (15,8)
    plt.grid(True)
    plt.plot(xs, ys, color="blue", label="Метод Рунге-Кутты 4го порядка " + str(h))
    plt.plot(xs, source_func(xs), color="red", label=f"Точное аналитическое решение Задачи c шагом " + str(h))
    plt.legend()
    plt.show()
    print("Максимальное отклонение от аналитического решения с шагом " + str(h), "равно : ", max_error(xs, ys, source_func))

In [5]:
ip_w.interact(comparison_source_with_Method_Runge_Kutta,  y0 = ip_w.fixed(Y0), a=ip_w.fixed(A), b=ip_w.fixed(B), h=ip_w.FloatSlider(min=0.01, max=0.1, step=0.01));

interactive(children=(FloatSlider(value=0.01, description='h', max=0.1, min=0.01, step=0.01), Output()), _dom_…

## Оценка погрешности с помощью правила Рунге

$$error = \frac{| y_h - y_{h/2}|}{2^p - 1}$$ 

In [6]:
def Runge_rule(ys1, ys2, p=1):
    return max(np.fabs(ys1 - ys2))/(2**p - 1)

In [7]:
def Runge_Kutta_estimation_error_Ruge_depending_on_h(y0,a,b,h):
    _, ys1 = Method_Runge_Kutta(func, y0, a, b, h)
    _, ys2 = Method_Runge_Kutta(func, Y0, A, B, h/2)
    if(len(ys1)!=len(ys2[::2])):
        ys1=np.delete(ys1,len(ys1)-1,0)
    err1 = Runge_rule(ys1, ys2[::2],5)
    print(f" Ошибка по Правилу рунге для шага {h} равна {round(err1, 7)}" )

In [8]:
ip_w.interact(Runge_Kutta_estimation_error_Ruge_depending_on_h,  y0 = ip_w.fixed(Y0), a=ip_w.fixed(A), b=ip_w.fixed(B), p=ip_w.fixed(1), h=ip_w.FloatSlider(min=0.01, max=0.1, step=0.01));

interactive(children=(FloatSlider(value=0.01, description='h', max=0.1, min=0.01, step=0.01), Output()), _dom_…

Ошибка по Правило рунге для шага 0.01 равна 0.0000705<br>
Ошибка по Правило рунге для шага 0.1 равна 0.0007902

## Нахождение решения с помощью библиотеки scipy

In [9]:
from scipy.integrate import odeint as ODE

In [10]:
g = lambda y, x : func(x,y)
def scipy_ODE(func, y0, a, b, h):
    xs = np.arange(a, b+h, h)
    ys = ODE(func, y0, xs)
    return xs, ys

In [11]:
def comparison_source_with_Scipy(y0, a, b, h):
    xs, ys = scipy_ODE(g, Y0, A, B, h)
    plt.rcParams["figure.figsize"] = (15,8)
    plt.grid(True)
    plt.plot(xs, ys, color="blue", label="Решение с помощью модуля scipy " + str(h))
    plt.plot(xs, source_func(xs), color="red", label=f"Точное аналитическое решение Задачи c шагом " + str(h))
    plt.legend()
    plt.show()    
    print("Максимальное отклонение от аналитического решения с шагом " + str(h), "равно : ", max_error(xs, ys.flat, source_func))


In [12]:
ip_w.interact(comparison_source_with_Scipy,  y0 = ip_w.fixed(Y0), a=ip_w.fixed(A), b=ip_w.fixed(B), h=ip_w.FloatSlider(min=0.01, max=0.1, step=0.01));

interactive(children=(FloatSlider(value=0.01, description='h', max=0.1, min=0.01, step=0.01), Output()), _dom_…

Значения от аналитического решения отличаются на 10^(-7)